In [1]:
import requests
import pandas as pd

# BioMart Query URL
biomart_url = "http://www.ensembl.org/biomart/martservice?query="

# XML Query for Ensembl -> Gene Symbol Mapping
query_xml = """<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE Query>
<Query virtualSchemaName = "default" formatter = "TSV" header = "1" uniqueRows = "1" count = "" datasetConfigVersion = "0.6">
    <Dataset name = "hsapiens_gene_ensembl" interface = "default" >
        <Attribute name = "ensembl_gene_id" />
        <Attribute name = "external_gene_name" />
    </Dataset>
</Query>"""

# Encode XML Query
query_encoded = requests.utils.quote(query_xml)

# Send request
response = requests.get(biomart_url + query_encoded)

# Check if request was successful
if response.status_code != 200:
    raise Exception(f"❌ BioMart query failed! Status Code: {response.status_code}")

# Convert response to list of lines
data = response.text.strip().split("\n")

# Check if response is empty
if len(data) <= 1:
    raise ValueError("❌ No data returned from BioMart. Check the query!")

# Extract the actual header from BioMart response
header = data[0].split("\t")

# **Fix: Adjust header names based on BioMart's response**
expected_headers = ["Gene stable ID", "Gene name"]
if header != expected_headers:
    raise ValueError(f"❌ Unexpected BioMart response format! Headers: {header}")

# Convert response into DataFrame with corrected column names
df_biomart = pd.DataFrame([x.split("\t") for x in data[1:] if "\t" in x], columns=["ensembl_gene_id", "gene_symbol"])

# Drop any empty values (sometimes there are empty rows)
df_biomart = df_biomart.dropna()

# Display first few rows
print("✅ BioMart Mapping Preview:")
print(df_biomart.head())

# Save mapped genes
df_biomart.to_csv("biomart_gene_mapping.csv", index=False)

print(f"✅ BioMart mapping completed and saved! {len(df_biomart)} genes mapped.")


✅ BioMart Mapping Preview:
   ensembl_gene_id gene_symbol
0  ENSG00000210049       MT-TF
1  ENSG00000211459     MT-RNR1
2  ENSG00000210077       MT-TV
3  ENSG00000210082     MT-RNR2
4  ENSG00000209082      MT-TL1
✅ BioMart mapping completed and saved! 86401 genes mapped.


In [2]:
import pandas as pd
import scanpy as sc
import numpy as np

# Load dataset
adata = sc.read_h5ad('/data/cellular_aging/dataset/AIDA.h5ad')

# Load BioMart gene mapping
df_biomart = pd.read_csv("biomart_gene_mapping.csv")
mapped_gene_dict = dict(zip(df_biomart["ensembl_gene_id"], df_biomart["gene_symbol"]))

# Extract Ensembl IDs (remove version numbers)
adata.var["ensembl_id"] = adata.var_names.str.split(".").str[0]

# **Total Genes & Expression Before Mapping**
total_genes_before = adata.shape[1]
total_expression_before = adata.X.sum()

# **Preview First 5 Genes Before Mapping**
print("🔹 **First 5 Genes BEFORE Mapping:**")
print(adata.var_names[:5])

# **Expression of First 5 Genes BEFORE Mapping**
expression_before = adata[:, :5].X.sum(axis=0)
if hasattr(expression_before, "A1"):  # Handle sparse matrices
    expression_before = expression_before.A1
else:
    expression_before = expression_before.flatten()

before_mapping_df = pd.DataFrame({"Total Expression": expression_before}, index=adata.var_names[:5])
print(before_mapping_df)

# **Apply BioMart Mapping**
adata.var["gene_symbol"] = adata.var["ensembl_id"].map(mapped_gene_dict)

# **Drop Unmapped Genes**
adata_filtered = adata[:, adata.var["gene_symbol"].notna()].copy()

# **Total Genes & Expression After Mapping**
total_genes_after = adata_filtered.shape[1]
total_expression_after = adata_filtered.X.sum()

# **Compute Mapping Coverage**
gene_coverage = (total_genes_after / total_genes_before) * 100
expression_coverage = (total_expression_after / total_expression_before) * 100

# **Preview First 5 Genes After Mapping**
print("\n🔹 **First 5 Genes AFTER Mapping:**")
print(adata_filtered.var["gene_symbol"][:5])

# **Expression of First 5 Genes AFTER Mapping**
expression_after = adata_filtered[:, :5].X.sum(axis=0)
if hasattr(expression_after, "A1"):  
    expression_after = expression_after.A1
else:
    expression_after = expression_after.flatten()

after_mapping_df = pd.DataFrame({"Total Expression": expression_after}, index=adata_filtered.var["gene_symbol"][:5])
print(after_mapping_df)

# **Print Summary**
print("\n📊 **Gene Mapping Coverage Summary:**")
print(f"Total Genes Before Mapping: {total_genes_before}")
print(f"Total Genes After Mapping: {total_genes_after}")
print(f"Total Expression Before Mapping: {total_expression_before:.2f}")
print(f"Total Expression After Mapping: {total_expression_after:.2f}")
print(f"Gene Mapping Coverage: {gene_coverage:.2f}%")
print(f"Expression Mapping Coverage: {expression_coverage:.2f}%")


🔹 **First 5 Genes BEFORE Mapping:**
Index(['ENSG00000243485', 'ENSG00000237613', 'ENSG00000186092',
       'ENSG00000238009', 'ENSG00000239945'],
      dtype='object')
                 Total Expression
ENSG00000243485         19.891203
ENSG00000237613         16.083389
ENSG00000186092          1.224736
ENSG00000238009        996.870972
ENSG00000239945          1.999470

🔹 **First 5 Genes AFTER Mapping:**
ENSG00000243485    MIR1302-2HG
ENSG00000237613        FAM138A
ENSG00000186092          OR4F5
ENSG00000284733         OR4F29
ENSG00000284662         OR4F16
Name: gene_symbol, dtype: object
             Total Expression
gene_symbol                  
MIR1302-2HG         19.891203
FAM138A             16.083389
OR4F5                1.224736
OR4F29               1.379211
OR4F16               0.000000

📊 **Gene Mapping Coverage Summary:**
Total Genes Before Mapping: 36161
Total Genes After Mapping: 25083
Total Expression Before Mapping: 2888279552.00
Total Expression After Mapping: 2863954176

In [3]:
import pandas as pd
import scanpy as sc
import mygene
import numpy as np

# 🔹 Load dataset (already mapped using BioMart)
adata = sc.read_h5ad('/data/cellular_aging/dataset/AIDA.h5ad')

# 🔹 Load BioMart mapping file
df_biomart = pd.read_csv("biomart_gene_mapping.csv")
mapped_gene_dict = dict(zip(df_biomart["ensembl_gene_id"], df_biomart["gene_symbol"]))

# 🔹 Extract Ensembl IDs correctly
adata.var["ensembl_id"] = adata.var_names.str.split(".").str[0]

# 🔹 Ensure all genes have Ensembl IDs
if "ensembl_id" not in adata.var.columns:
    adata.var["ensembl_id"] = adata.var_names

# **Total Genes & Expression Before Mapping**
total_genes_before = adata.shape[1]
total_expression_before = adata.X.sum()

# **Preview First 5 Genes Before Mapping**
print("🔹 **First 5 Genes BEFORE Mapping:**")
print(adata.var_names[:5])

# **Expression of First 5 Genes BEFORE Mapping**
expression_before = adata[:, :5].X.sum(axis=0)
if hasattr(expression_before, "A1"):  # Handle sparse matrices
    expression_before = expression_before.A1
else:
    expression_before = expression_before.flatten()

before_mapping_df = pd.DataFrame({"Total Expression": expression_before}, index=adata.var_names[:5])
print(before_mapping_df)

# 🔹 Apply BioMart Mapping
adata.var["gene_symbol"] = adata.var["ensembl_id"].map(mapped_gene_dict)

# 🔹 Find unmatched genes (not mapped via BioMart)
unmatched_genes = adata.var[adata.var["gene_symbol"].isna()]["ensembl_id"].tolist()
print(f"❌ Unmatched Genes After BioMart: {len(unmatched_genes)}")
print("🔍 Example Unmatched Genes:", unmatched_genes[:10])

# **Total Genes & Expression After BioMart Mapping**
adata_biomart_filtered = adata[:, adata.var["gene_symbol"].notna()].copy()
total_genes_after_biomart = adata_biomart_filtered.shape[1]
total_expression_after_biomart = adata_biomart_filtered.X.sum()

# ✅ Query MyGene.info for missing gene symbols
mg = mygene.MyGeneInfo()
query_results = mg.querymany(
    unmatched_genes, 
    scopes="ensembl.gene",
    fields="symbol",
    species="human",
    as_dataframe=True,
    df_index=True,
)

# ✅ Clean up results (remove duplicates, missing values)
query_results = query_results.dropna(subset=["symbol"])
query_results = query_results[~query_results.symbol.str.contains("\|", na=False)]
query_results_dict = query_results["symbol"].to_dict()

print(f"✅ Newly mapped genes from MyGene.info: {len(query_results_dict)}")
print("🔍 Example newly mapped genes:", list(query_results_dict.keys())[:10])

# ✅ Apply MyGene.info updates correctly
adata.var.loc[adata.var["ensembl_id"].isin(query_results_dict), "gene_symbol"] = \
    adata.var["ensembl_id"].map(query_results_dict)

# 🔹 Count remaining unmatched genes
still_unmatched = adata.var[adata.var["gene_symbol"].isna()]["ensembl_id"].tolist()
print(f"❌ Genes still unmatched after MyGene.info: {len(still_unmatched)}")
print("🔍 Example unmatched genes:", still_unmatched[:10])

# **Total Genes & Expression After MyGene.info Mapping**
adata_final_filtered = adata[:, adata.var["gene_symbol"].notna()].copy()
total_genes_after_mygene = adata_final_filtered.shape[1]
total_expression_after_mygene = adata_final_filtered.X.sum()

# **Compute Mapping Coverage**
gene_coverage_biomart = (total_genes_after_biomart / total_genes_before) * 100
expression_coverage_biomart = (total_expression_after_biomart / total_expression_before) * 100

gene_coverage_final = (total_genes_after_mygene / total_genes_before) * 100
expression_coverage_final = (total_expression_after_mygene / total_expression_before) * 100

# **Preview First 5 Genes After MyGene.info Mapping**
print("\n🔹 **First 5 Genes AFTER MyGene.info Mapping:**")
print(adata_final_filtered.var["gene_symbol"][:5])

# **Expression of First 5 Genes AFTER MyGene.info Mapping**
expression_after = adata_final_filtered[:, :5].X.sum(axis=0)
if hasattr(expression_after, "A1"):  
    expression_after = expression_after.A1
else:
    expression_after = expression_after.flatten()

after_mapping_df = pd.DataFrame({"Total Expression": expression_after}, index=adata_final_filtered.var["gene_symbol"][:5])
print(after_mapping_df)

# ✅ Save the final updated gene mapping
adata.var[["ensembl_id", "gene_symbol"]].to_csv("final_gene_mapping.csv", index=False)
print("✅ Final gene mapping saved as 'final_gene_mapping.csv'.")

# **Print Summary**
print("\n📊 **Gene Mapping Coverage Summary:**")
print(f"Total Genes Before Mapping: {total_genes_before}")
print(f"Total Genes After BioMart Mapping: {total_genes_after_biomart}")
print(f"Total Genes After MyGene.info Mapping: {total_genes_after_mygene}")
print(f"Total Expression Before Mapping: {total_expression_before:.2f}")
print(f"Total Expression After BioMart Mapping: {total_expression_after_biomart:.2f}")
print(f"Total Expression After MyGene.info Mapping: {total_expression_after_mygene:.2f}")
print(f"Gene Mapping Coverage After BioMart: {gene_coverage_biomart:.2f}%")
print(f"Expression Mapping Coverage After BioMart: {expression_coverage_biomart:.2f}%")
print(f"Final Gene Mapping Coverage After MyGene.info: {gene_coverage_final:.2f}%")
print(f"Final Expression Mapping Coverage After MyGene.info: {expression_coverage_final:.2f}%")


🔹 **First 5 Genes BEFORE Mapping:**
Index(['ENSG00000243485', 'ENSG00000237613', 'ENSG00000186092',
       'ENSG00000238009', 'ENSG00000239945'],
      dtype='object')
                 Total Expression
ENSG00000243485         19.891203
ENSG00000237613         16.083389
ENSG00000186092          1.224736
ENSG00000238009        996.870972
ENSG00000239945          1.999470
❌ Unmatched Genes After BioMart: 11078
🔍 Example Unmatched Genes: ['ENSG00000238009', 'ENSG00000239945', 'ENSG00000239906', 'ENSG00000241860', 'ENSG00000241599', 'ENSG00000286448', 'ENSG00000236601', 'ENSG00000235146', 'ENSG00000229905', 'ENSG00000272438']


Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
23 input query terms found dup hits:	[('ENSG00000228044', 2), ('ENSG00000226506', 2), ('ENSG00000261600', 2), ('ENSG00000234162', 2), ('E
924 input query terms found no hit:	['ENSG00000238009', 'ENSG00000230699', 'ENSG00000241180', 'ENSG00000236948', 'ENSG00000226849', 'ENS


✅ Newly mapped genes from MyGene.info: 1335
🔍 Example newly mapped genes: ['ENSG00000287356', 'ENSG00000287396', 'ENSG00000259961', 'ENSG00000282740', 'ENSG00000238142', 'ENSG00000227066', 'ENSG00000284641', 'ENSG00000285802', 'ENSG00000237429', 'ENSG00000235143']
❌ Genes still unmatched after MyGene.info: 9743
🔍 Example unmatched genes: ['ENSG00000238009', 'ENSG00000239945', 'ENSG00000239906', 'ENSG00000241860', 'ENSG00000241599', 'ENSG00000286448', 'ENSG00000236601', 'ENSG00000235146', 'ENSG00000229905', 'ENSG00000272438']

🔹 **First 5 Genes AFTER MyGene.info Mapping:**
ENSG00000243485    MIR1302-2HG
ENSG00000237613        FAM138A
ENSG00000186092          OR4F5
ENSG00000284733         OR4F29
ENSG00000284662         OR4F16
Name: gene_symbol, dtype: object
             Total Expression
gene_symbol                  
MIR1302-2HG         19.891203
FAM138A             16.083389
OR4F5                1.224736
OR4F29               1.379211
OR4F16               0.000000
✅ Final gene mapping sa

In [11]:
import json
import pandas as pd
import scanpy as sc
import numpy as np

# 🔹 Load dataset (already mapped using BioMart & MyGene.info)
adata = sc.read_h5ad('/data/cellular_aging/dataset/AIDA.h5ad')

# 🔹 Load the FINAL gene mapping file (from BioMart + MyGene.info)
df_final_mapping = pd.read_csv("final_gene_mapping.csv")
final_gene_dict = dict(zip(df_final_mapping["ensembl_id"], df_final_mapping["gene_symbol"]))

# 🔹 Ensure Ensembl IDs are extracted correctly
adata.var["ensembl_id"] = adata.var_names.str.split(".").str[0]

# 🔹 Apply the final mapping to adata
adata.var["gene_symbol"] = adata.var["ensembl_id"].map(final_gene_dict)

# **Total Genes & Expression Before Vocab Filtering**
total_genes_before_vocab = adata.shape[1]
total_expression_before_vocab = adata.X.sum()

# 🔹 Load the vocab.json file
vocab_path = "/data/cellular_aging/references/scGPT_human_pretrained_model/vocab.json"
with open(vocab_path, "r") as f:
    vocab = json.load(f)

# 🔹 Convert vocab keys (gene names) into a set
vocab_genes = set(vocab.keys())

# 🔹 Find unmatched genes (genes mapped but NOT in vocab.json)
mapped_genes = set(adata.var["gene_symbol"].dropna().unique())
unmatched_vocab_genes = mapped_genes - vocab_genes

# **Print Matched & Unmatched Stats**
print(f"✅ Matched Genes in vocab.json: {len(mapped_genes - unmatched_vocab_genes)} / {len(mapped_genes)}")
print(f"❌ Unmatched Genes Not in vocab.json: {len(unmatched_vocab_genes)}")
print("🔍 Example unmatched genes:", list(unmatched_vocab_genes)[:10])

# **Filter dataset to keep only genes in vocab.json**
adata_filtered_vocab = adata[:, adata.var["gene_symbol"].isin(vocab_genes)].copy()

# **Total Genes & Expression After Vocab Filtering**
total_genes_after_vocab = adata_filtered_vocab.shape[1]
total_expression_after_vocab = adata_filtered_vocab.X.sum()

# **Compute Coverage & Data Loss**
gene_coverage_vocab = (total_genes_after_vocab / total_genes_before_vocab) * 100
expression_coverage_vocab = (total_expression_after_vocab / total_expression_before_vocab) * 100

# 🔹 Print Final Summary
print("\n📊 **Final Gene Mapping & Expression Coverage Summary:**")
print(f"Total Genes Before Vocab Filtering: {total_genes_before_vocab}")
print(f"Total Genes After Vocab Filtering: {total_genes_after_vocab}")
print(f"Total Expression Before Vocab Filtering: {total_expression_before_vocab:.2f}")
print(f"Total Expression After Vocab Filtering: {total_expression_after_vocab:.2f}")
print(f"Gene Mapping Coverage After Vocab Filtering: {gene_coverage_vocab:.2f}%")
print(f"Expression Mapping Coverage After Vocab Filtering: {expression_coverage_vocab:.2f}%")

# 🔹 Save the filtered dataset
adata_filtered_vocab.write("adata_filtered_vocab.h5ad")
print("✅ Final filtered dataset saved as 'adata_filtered_vocab.h5ad'.")


✅ Matched Genes in vocab.json: 23791 / 26406
❌ Unmatched Genes Not in vocab.json: 2615
🔍 Example unmatched genes: ['CIST1', 'LOC107986768', 'LOC124903183', 'GTF2A1-AS1', 'SPACA6-AS1', 'CYB561D2', 'LOC105371401', 'ARK2C', 'LOC124903670', 'SPMIP1']

📊 **Final Gene Mapping & Expression Coverage Summary:**
Total Genes Before Vocab Filtering: 36161
Total Genes After Vocab Filtering: 23794
Total Expression Before Vocab Filtering: 2888279552.00
Total Expression After Vocab Filtering: 2841698816.00
Gene Mapping Coverage After Vocab Filtering: 65.80%
Expression Mapping Coverage After Vocab Filtering: 98.39%
✅ Final filtered dataset saved as 'adata_filtered_vocab.h5ad'.


In [13]:
import json
import pandas as pd
import scanpy as sc
import mygene

# 🔹 Load dataset (already mapped using BioMart & MyGene.info)
adata = sc.read_h5ad('/data/cellular_aging/dataset/AIDA.h5ad')

# 🔹 Load the FINAL gene mapping file (from BioMart + MyGene.info)
df_final_mapping = pd.read_csv("final_gene_mapping.csv")
final_gene_dict = dict(zip(df_final_mapping["ensembl_id"], df_final_mapping["gene_symbol"]))

# 🔹 Ensure Ensembl IDs are extracted correctly
adata.var["ensembl_id"] = adata.var_names.str.split(".").str[0]

# 🔹 Apply the final mapping to adata
adata.var["gene_symbol"] = adata.var["ensembl_id"].map(final_gene_dict)

# 🔹 Load the vocab.json file
vocab_path = "/data/cellular_aging/references/scGPT_human_pretrained_model/vocab.json"
with open(vocab_path, "r") as f:
    vocab = json.load(f)

# 🔹 Convert vocab keys (gene names) into a set
vocab_genes = set(vocab.keys())

# 🔹 Find unmatched genes (genes mapped but NOT in vocab.json)
mapped_genes = set(adata.var["gene_symbol"].dropna().unique())
unmatched_vocab_genes = mapped_genes - vocab_genes

print(f"✅ Matched Genes in vocab.json: {len(mapped_genes - unmatched_vocab_genes)} / {len(mapped_genes)}")
print(f"❌ Unmatched Genes Not in vocab.json: {len(unmatched_vocab_genes)}")
print("🔍 Example unmatched genes:", list(unmatched_vocab_genes)[:10])

# **Filter dataset to keep only genes in vocab.json**
adata_filtered_vocab = adata[:, adata.var["gene_symbol"].isin(vocab_genes)].copy()
print(f"✅ Filtered dataset shape after vocab filtering: {adata_filtered_vocab.shape}")

# **Compute Final Coverage Statistics**
total_genes_before_vocab = adata.shape[1]
total_genes_after_vocab = adata_filtered_vocab.shape[1]
total_expression_before_vocab = adata.X.sum()
total_expression_after_vocab = adata_filtered_vocab.X.sum()

gene_coverage_vocab = (total_genes_after_vocab / total_genes_before_vocab) * 100
expression_coverage_vocab = (total_expression_after_vocab / total_expression_before_vocab) * 100

# **Final Summary**
print("\n📊 **Final Gene Mapping & Expression Coverage Summary:**")
print(f"Total Genes Before Vocab Filtering: {total_genes_before_vocab}")
print(f"Total Genes After Vocab Filtering: {total_genes_after_vocab}")
print(f"Total Expression Before Vocab Filtering: {total_expression_before_vocab:.2f}")
print(f"Total Expression After Vocab Filtering: {total_expression_after_vocab:.2f}")
print(f"Gene Mapping Coverage After Vocab Filtering: {gene_coverage_vocab:.2f}%")
print(f"Expression Mapping Coverage After Vocab Filtering: {expression_coverage_vocab:.2f}%")

# 🔹 Save the filtered dataset
adata_filtered_vocab.write("adata_filtered_vocab.h5ad")
print("✅ Final filtered dataset saved as 'adata_filtered_vocab.h5ad'.")

### 🔎 1️⃣ Find the top removed genes by expression
unmatched_genes_df = adata.var.loc[adata.var["gene_symbol"].isin(unmatched_vocab_genes)]
removed_expression = adata[:, unmatched_genes_df.index].X.sum(axis=0)

if hasattr(removed_expression, "A1"):
    removed_expression = removed_expression.A1

removed_genes_df = pd.DataFrame({
    "gene_symbol": unmatched_genes_df["gene_symbol"],
    "total_expression": removed_expression
}).sort_values(by="total_expression", ascending=False)

print("\n🔍 **Top Removed Genes by Expression:**")
print(removed_genes_df.head(10))

# 🔹 Save removed genes for inspection
removed_genes_df.to_csv("removed_genes_expression.csv", index=False)
print("✅ Removed genes with expression values saved as 'removed_genes_expression.csv'.")

### 🔎 2️⃣ Check if lost genes exist in HGNC database
mg = mygene.MyGeneInfo()
query_results = mg.querymany(
    list(unmatched_vocab_genes), 
    scopes="symbol",
    fields="name",
    species="human",
    as_dataframe=True,
    df_index=True,
)

query_results = query_results.dropna(subset=["name"])

print(f"✅ Found {len(query_results)} lost genes in HGNC database.")
print("🔍 Example found genes:", query_results.head(10))

query_results.to_csv("hgnc_found_lost_genes.csv", index=False)
print("✅ HGNC lookup results saved as 'hgnc_found_lost_genes.csv'.")

### 🔎 3️⃣ Compare removed genes with Ensembl mappings
df_ensembl_check = df_final_mapping[df_final_mapping["gene_symbol"].isin(unmatched_vocab_genes)]
print(f"✅ {len(df_ensembl_check)} removed genes were originally mapped from Ensembl.")

print("\n🔍 Example Ensembl-mapped lost genes:")
print(df_ensembl_check.head(10))

df_ensembl_check.to_csv("removed_genes_ensembl_mapping.csv", index=False)
print("✅ Ensembl-mapped lost genes saved as 'removed_genes_ensembl_mapping.csv'.")


✅ Matched Genes in vocab.json: 23791 / 26406
❌ Unmatched Genes Not in vocab.json: 2615
🔍 Example unmatched genes: ['CIST1', 'LOC107986768', 'LOC124903183', 'GTF2A1-AS1', 'SPACA6-AS1', 'CYB561D2', 'LOC105371401', 'ARK2C', 'LOC124903670', 'SPMIP1']
✅ Filtered dataset shape after vocab filtering: (1058909, 23794)

📊 **Final Gene Mapping & Expression Coverage Summary:**
Total Genes Before Vocab Filtering: 36161
Total Genes After Vocab Filtering: 23794
Total Expression Before Vocab Filtering: 2888279552.00
Total Expression After Vocab Filtering: 2841698816.00
Gene Mapping Coverage After Vocab Filtering: 65.80%
Expression Mapping Coverage After Vocab Filtering: 98.39%
✅ Final filtered dataset saved as 'adata_filtered_vocab.h5ad'.


Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed



🔍 **Top Removed Genes by Expression:**
                  gene_symbol  total_expression
ENSG00000109062        NHERF1      904557.68750
ENSG00000171159          BBLN      854871.12500
ENSG00000112739         PRP4K      536355.50000
ENSG00000265206      MIR142HG      498323.68750
ENSG00000243449        NICOL1      432258.65625
ENSG00000197536        CARINH      400893.84375
ENSG00000285437       POLR2J3      336505.65625
ENSG00000182831       HAPSTR1      330214.78125
ENSG00000167085          PHB1      329154.81250
ENSG00000285976  LOC128125822      311096.15625
✅ Removed genes with expression values saved as 'removed_genes_expression.csv'.


422 input query terms found dup hits:	[('CYP26C1-DT', 2), ('HIVEP2-DT', 2), ('CENPBD1P', 2), ('FRMD4A-AS1', 2), ('NUBPL-DT', 2), ('IGSF22-
2 input query terms found no hit:	['NUP107-DT', 'SMN-AS1']


✅ Found 3054 lost genes in HGNC database.
🔍 Example found genes:                     _id     _score  \
query                                
CIST1            729966  19.894098   
LOC107986768  107986768  27.165703   
LOC124903183  124903183  27.165703   
GTF2A1-AS1    101928504  27.165703   
SPACA6-AS1    102238594  27.165703   
CYB561D2          11068  18.761572   
LOC105371401  105371401  27.165703   
ARK2C            494470  18.213528   
LOC124903670  124903670  27.165703   
SPMIP1        100130705  18.875212   

                                                           name notfound  
query                                                                     
CIST1                   colon, intestine and stomach enriched 1      NaN  
LOC107986768                       uncharacterized LOC107986768      NaN  
LOC124903183                       uncharacterized LOC124903183      NaN  
GTF2A1-AS1                               GTF2A1 antisense RNA 1      NaN  
SPACA6-AS1                  

GTF code Analysis from Admera which is same as Biomart in terms of Percentage

In [4]:
import gzip
import pandas as pd

# Path to your GTF file
gtf_file = "/data/cellular_aging/references/scGPT_human_pretrained_model/Homo_sapiens.GRCh38.113.gtf.gz"

# Storage for extracted gene mappings
ensembl_to_symbol = {}

# Read GTF file line by line (skipping headers)
with gzip.open(gtf_file, 'rt') as f:
    for line in f:
        if line.startswith("#"):
            continue  # Skip comment lines
        fields = line.strip().split("\t")
        if fields[2] == "gene":  # Only process gene entries
            attributes = {kv.split(" ")[0]: kv.split(" ")[1].strip('";') for kv in fields[8].split("; ") if " " in kv}
            ensembl_id = attributes.get("gene_id", "")
            gene_symbol = attributes.get("gene_name", "")
            if ensembl_id and gene_symbol:
                ensembl_to_symbol[ensembl_id] = gene_symbol

# Convert mapping to DataFrame
df_gtf = pd.DataFrame(list(ensembl_to_symbol.items()), columns=["ensembl_gene_id", "gene_symbol"])

# Save mapping for later use
df_gtf.to_csv("gtf_gene_mapping.csv", index=False)

print(f"✅ Extracted {len(df_gtf)} gene mappings from GTF file.")
print(df_gtf.head())


✅ Extracted 42745 gene mappings from GTF file.
   ensembl_gene_id gene_symbol
0  ENSG00000142611      PRDM16
1  ENSG00000157911       PEX10
2  ENSG00000224340    RPL21P21
3  ENSG00000229280     EEF1DP6
4  ENSG00000142655       PEX14


In [5]:
import scanpy as sc

# Load dataset
adata = sc.read_h5ad('/data/cellular_aging/dataset/AIDA.h5ad')

# Load the GTF gene mapping file
df_gtf = pd.read_csv("gtf_gene_mapping.csv")

# Create a mapping dictionary
gtf_gene_dict = dict(zip(df_gtf["ensembl_gene_id"], df_gtf["gene_symbol"]))

# Extract Ensembl IDs from dataset
adata.var["ensembl_id"] = adata.var_names.str.split(".").str[0]

# Apply mapping
adata.var["gene_symbol"] = adata.var["ensembl_id"].map(gtf_gene_dict)

# Count mapped and unmatched genes
total_genes_before = adata.shape[1]
total_genes_after = adata.var["gene_symbol"].notna().sum()

# Compute coverage
gene_coverage = (total_genes_after / total_genes_before) * 100

print("\n📊 **Gene Mapping Coverage Summary using GTF:**")
print(f"Total Genes Before Mapping: {total_genes_before}")
print(f"Total Genes After Mapping: {total_genes_after}")
print(f"Gene Mapping Coverage: {gene_coverage:.2f}%")

# Save the updated gene mapping
adata.var[["ensembl_id", "gene_symbol"]].to_csv("final_gtf_gene_mapping.csv", index=False)
print("✅ Final gene mapping saved as 'final_gtf_gene_mapping.csv'.")



📊 **Gene Mapping Coverage Summary using GTF:**
Total Genes Before Mapping: 36161
Total Genes After Mapping: 25083
Gene Mapping Coverage: 69.36%
✅ Final gene mapping saved as 'final_gtf_gene_mapping.csv'.


In [7]:
import pandas as pd
import scanpy as sc
import numpy as np

# 🔹 Load dataset
adata = sc.read_h5ad('/data/cellular_aging/dataset/AIDA.h5ad')

# 🔹 Load the GTF gene mapping file
df_gtf = pd.read_csv("gtf_gene_mapping.csv")

# 🔹 Create a mapping dictionary
gtf_gene_dict = dict(zip(df_gtf["ensembl_gene_id"], df_gtf["gene_symbol"]))

# 🔹 Extract Ensembl IDs correctly
adata.var["ensembl_id"] = adata.var_names.str.split(".").str[0]

# **Total Genes & Expression Before Mapping**
total_genes_before = adata.shape[1]
total_expression_before = adata.X.sum()

# **Preview First 5 Genes Before Mapping**
print("🔹 **First 5 Genes BEFORE Mapping:**")
print(adata.var_names[:5])

# **Expression of First 5 Genes BEFORE Mapping**
expression_before = adata[:, :5].X.sum(axis=0)
if hasattr(expression_before, "A1"):  # Handle sparse matrices
    expression_before = expression_before.A1
else:
    expression_before = expression_before.flatten()

before_mapping_df = pd.DataFrame({"Total Expression": expression_before}, index=adata.var_names[:5])
print(before_mapping_df)

# **Apply GTF Mapping**
adata.var["gene_symbol"] = adata.var["ensembl_id"].map(gtf_gene_dict)

# **Drop Unmapped Genes**
adata_filtered = adata[:, adata.var["gene_symbol"].notna()].copy()

# **Total Genes & Expression After Mapping**
total_genes_after = adata_filtered.shape[1]
total_expression_after = adata_filtered.X.sum()

# **Compute Mapping Coverage**
gene_coverage = (total_genes_after / total_genes_before) * 100
expression_coverage = (total_expression_after / total_expression_before) * 100

# **Preview First 5 Genes After Mapping**
print("\n🔹 **First 5 Genes AFTER Mapping:**")
print(adata_filtered.var["gene_symbol"][:5])

# **Expression of First 5 Genes AFTER Mapping**
expression_after = adata_filtered[:, :5].X.sum(axis=0)
if hasattr(expression_after, "A1"):  
    expression_after = expression_after.A1
else:
    expression_after = expression_after.flatten()

after_mapping_df = pd.DataFrame({"Total Expression": expression_after}, index=adata_filtered.var["gene_symbol"][:5])
print(after_mapping_df)

# **Print Summary**
print("\n📊 **Gene Mapping Coverage Summary using GTF:**")
print(f"Total Genes Before Mapping: {total_genes_before}")
print(f"Total Genes After GTF Mapping: {total_genes_after}")
print(f"Total Expression Before Mapping: {total_expression_before:.2f}")
print(f"Total Expression After GTF Mapping: {total_expression_after:.2f}")
print(f"Gene Mapping Coverage: {gene_coverage:.2f}%")
print(f"Expression Mapping Coverage: {expression_coverage:.2f}%")

# ✅ Save the final gene mapping
adata.var[["ensembl_id", "gene_symbol"]].to_csv("final_gtf_gene_mapping.csv", index=False)
print("✅ Final gene mapping saved as 'final_gtf_gene_mapping.csv'.")


🔹 **First 5 Genes BEFORE Mapping:**
Index(['ENSG00000243485', 'ENSG00000237613', 'ENSG00000186092',
       'ENSG00000238009', 'ENSG00000239945'],
      dtype='object')
                 Total Expression
ENSG00000243485         19.891203
ENSG00000237613         16.083389
ENSG00000186092          1.224736
ENSG00000238009        996.870972
ENSG00000239945          1.999470

🔹 **First 5 Genes AFTER Mapping:**
ENSG00000243485    MIR1302-2HG
ENSG00000237613        FAM138A
ENSG00000186092          OR4F5
ENSG00000284733         OR4F29
ENSG00000284662         OR4F16
Name: gene_symbol, dtype: object
             Total Expression
gene_symbol                  
MIR1302-2HG         19.891203
FAM138A             16.083389
OR4F5                1.224736
OR4F29               1.379211
OR4F16               0.000000

📊 **Gene Mapping Coverage Summary using GTF:**
Total Genes Before Mapping: 36161
Total Genes After GTF Mapping: 25083
Total Expression Before Mapping: 2888279552.00
Total Expression After GTF M

In [3]:
import json
import pandas as pd
from pathlib import Path

# Set the correct vocab.json path
vocab_path = Path("/data/cellular_aging/references/scGPT_human_pretrained_model/vocab.json")

# Load vocab.json
with open(vocab_path, "r") as f:
    vocab = json.load(f)

# Convert vocab keys (gene names) into a set
vocab_genes = set(vocab.keys())

# Load your mapped gene dataset
df_mapped = pd.read_csv("biomart_gene_mapping.csv")  # Ensure this file is in the current directory
mapped_genes = set(df_mapped["gene_symbol"])  # Extract mapped gene symbols

# Find how many genes match
matching_genes = mapped_genes.intersection(vocab_genes)
non_matching_genes = mapped_genes - vocab_genes

print(f"✅ Matched Genes: {len(matching_genes)} / {len(mapped_genes)} ({(len(matching_genes)/len(mapped_genes)) * 100:.2f}%)")
print(f"❌ Unmatched Genes Example: {list(non_matching_genes)[:10]}")


✅ Matched Genes: 38626 / 41150 (93.87%)
❌ Unmatched Genes Example: ['AFTPH-DT', 'DUT-AS1', 'RPL29P9', 'TNPO3P2', 'GOLGA6L26', 'POLR2J3', 'SDHBP1', 'TMEM276-ZFTRAF1', 'NEMP2-DT', 'MIR1539']


In [4]:
import json
import pandas as pd

# Load the vocab.json file
vocab_path = "/data/cellular_aging/references/scGPT_human_pretrained_model/vocab.json"

with open(vocab_path, "r") as f:
    vocab = json.load(f)

# Convert vocab keys (gene names) into a set
vocab_genes = set(vocab.keys())

# Load the mapped gene file (biomart_gene_mapping.csv)
df_mapped = pd.read_csv("biomart_gene_mapping.csv")

# Extract gene symbols from the mapped dataset
mapped_genes = set(df_mapped["gene_symbol"].dropna().unique())

# Find genes that are in the dataset but NOT in vocab.json
unmatched_genes = list(mapped_genes - vocab_genes)

# Display the first few unmatched genes
print(f"❌ Unmatched Genes: {len(unmatched_genes)} / {len(mapped_genes)}")
print(unmatched_genes[:50])  # Show first 50 unmatched genes

# Save the unmatched genes to a file for manual inspection
pd.DataFrame(unmatched_genes, columns=["Unmatched_Genes"]).to_csv("unmatched_genes.csv", index=False)
print("✅ Unmatched genes saved as 'unmatched_genes.csv'.")


❌ Unmatched Genes: 2523 / 41149
['AFTPH-DT', 'DUT-AS1', 'RPL29P9', 'TNPO3P2', 'GOLGA6L26', 'POLR2J3', 'SDHBP1', 'TMEM276-ZFTRAF1', 'NEMP2-DT', 'MIR1539', 'RLIG1', 'RPL31P29', 'CHD1-DT', 'SYNM-AS2', 'RPL7AP48', 'RPL7AP55', 'LINC02901', 'NBPF7P', 'TARS1-DT', 'DTWD1P1', 'RPS18P8', 'HEATR3-AS1', 'ACTR3-AS1', 'RPL21P82', 'RPSA2', 'TMEM69P2', 'FAM13B-AS1', 'EVA1CP2', 'FAM88D', 'ZNF691-DT', 'U7', 'RPS26P22', 'UAP1-DT', 'LINC03013', 'TRPC6P4', 'SKAP1-AS2', 'C9orf78P1', 'SPTBN1-AS2', 'ZNF277-AS1', 'C3orf86P', 'KIF23-AS1', 'TEX28P3', 'TMEM217B', 'TDRD6-AS1', 'NPIPA6', 'ADAD1P1', 'PENK-AS1', 'OR52Z1P', 'DPPA4P2', 'GPR15LG']
✅ Unmatched genes saved as 'unmatched_genes.csv'.


In [6]:
import pandas as pd
import scanpy as sc

# 🔹 Load dataset (already mapped using BioMart)
adata = sc.read_h5ad('/data/cellular_aging/dataset/AIDA.h5ad')

# 🔹 Load BioMart mapping file
df_biomart = pd.read_csv("biomart_gene_mapping.csv")
mapped_gene_dict = dict(zip(df_biomart["ensembl_gene_id"], df_biomart["gene_symbol"]))

# 🔹 Extract Ensembl IDs correctly
adata.var["ensembl_id"] = adata.var_names.str.split(".").str[0]

# 🔹 Verify if the column is created
print("Available columns in adata.var:", adata.var.columns.tolist())

# 🔹 Ensure all genes have Ensembl IDs
if "ensembl_id" not in adata.var.columns:
    adata.var["ensembl_id"] = adata.var_names

# 🔹 Map gene symbols
adata.var["gene_symbol"] = adata.var["ensembl_id"].map(mapped_gene_dict)

# 🔹 Find unmatched genes (not mapped via BioMart)
unmatched_genes = adata.var[adata.var["gene_symbol"].isna()]["ensembl_id"].tolist()
print(f"❌ Unmatched Genes After BioMart: {len(unmatched_genes)}")
print("🔍 Example Unmatched Genes:", unmatched_genes[:10])


Available columns in adata.var: ['feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype', 'feature_length', 'feature_type', 'ensembl_id']
❌ Unmatched Genes After BioMart: 11078
🔍 Example Unmatched Genes: ['ENSG00000238009', 'ENSG00000239945', 'ENSG00000239906', 'ENSG00000241860', 'ENSG00000241599', 'ENSG00000286448', 'ENSG00000236601', 'ENSG00000235146', 'ENSG00000229905', 'ENSG00000272438']


In [5]:
import scanpy as sc

# Load the updated dataset
updated_adata_path = "/data/cellular_aging/preprocessed_batches/binned_sample_updated.h5ad"
updated_adata = sc.read_h5ad(updated_adata_path)

# Check if there are still missing gene IDs
missing_after_update = [g for g in updated_adata.var_names if g not in gtf_gene_dict]

print(f"✅ Remaining missing genes after update: {len(missing_after_update)}")
print(f"📌 Sample missing genes (if any): {missing_after_update[:10]}")


✅ Remaining missing genes after update: 948
📌 Sample missing genes (if any): ['ENSG00000238009', 'ENSG00000230699', 'ENSG00000241180', 'ENSG00000236948', 'ENSG00000226849', 'ENSG00000272482', 'ENSG00000224621', 'ENSG00000234166', 'ENSG00000261135', 'ENSG00000264443']


In [8]:
import json
import pandas as pd
import gzip

# --- Load vocab.json and extract gene symbols ---
vocab_path = "/data/cellular_aging/references/scGPT_human_pretrained_model/vocab.json"
with open(vocab_path, "r") as f:
    vocab = json.load(f)
vocab_gene_symbols = set(vocab.keys())
print(f"Total gene symbols in vocab.json: {len(vocab_gene_symbols)}")
print("Sample vocab gene symbols:", list(vocab_gene_symbols)[:10])

# --- Extract gene symbols from the GTF file ---
gtf_path = "/data/cellular_aging/references/gencode.v47.annotation.gtf.gz"
gtf_gene_dict = {}  # mapping: Ensembl ID (without version) -> Gene Symbol

with gzip.open(gtf_path, "rt") as f:
    for line in f:
        if line.startswith("#"):
            continue
        fields = line.strip().split("\t")
        if fields[2] == "gene":
            attributes = fields[8]
            try:
                gene_id = attributes.split('gene_id "')[1].split('"')[0].split(".")[0]
            except IndexError:
                continue
            if 'gene_name "' in attributes:
                gene_symbol = attributes.split('gene_name "')[1].split('"')[0]
            else:
                gene_symbol = gene_id
            gtf_gene_dict[gene_id] = gene_symbol

gtf_gene_symbols = set(gtf_gene_dict.values())
print(f"Total gene symbols in GTF file: {len(gtf_gene_symbols)}")
print("Sample gene symbols from GTF:", list(gtf_gene_symbols)[:10])

# --- Compute the intersection ---
common_genes = vocab_gene_symbols.intersection(gtf_gene_symbols)
print(f"Common gene symbols between vocab.json and GTF: {len(common_genes)}")
print("Sample common gene symbols:", list(common_genes)[:10])


Total gene symbols in vocab.json: 60697
Sample vocab gene symbols: ['OR8S1', 'LL0XNC01-7P3.1', 'AC142528.1', 'DENND1C', 'RP11-373A6.2', 'SPRR3', 'RN7SL300P', 'RP5-1171I10.5', 'TBCAP3', 'LINC00375']
Total gene symbols in GTF file: 77114
Sample gene symbols from GTF: ['SPRR3', 'OR8S1', 'DENND1C', 'ENSG00000251391', 'RN7SL300P', 'TBCAP3', 'LINC00375', 'OR4K5', 'PPIAP21', 'ENSG00000272906']
Common gene symbols between vocab.json and GTF: 38621
Sample common gene symbols: ['OR8S1', 'SPRR3', 'DENND1C', 'RN7SL300P', 'TBCAP3', 'LINC00375', 'OR4K5', 'PPIAP21', 'SAGE3P', 'LINC01838']


In [9]:
import gzip
import json
import pandas as pd
import scanpy as sc

# -----------------------------
# Function to extract gene mapping from a GTF file
# -----------------------------
def extract_gene_mapping_from_gtf(gtf_file):
    """
    Extracts a dictionary mapping Ensembl gene IDs (stripped of version numbers)
    to gene symbols from a GTF file.
    """
    gene_mapping = {}
    with gzip.open(gtf_file, "rt") as f:
        for line in f:
            if line.startswith("#"):
                continue
            fields = line.strip().split("\t")
            if fields[2] != "gene":
                continue
            attributes = fields[-1]
            try:
                gene_id = attributes.split('gene_id "')[1].split('"')[0]
            except IndexError:
                continue
            # Remove version (e.g., ENSG00000290825.2 -> ENSG00000290825)
            gene_id_clean = gene_id.split(".")[0]
            # Extract gene_name if available; otherwise use gene_id
            if 'gene_name "' in attributes:
                gene_symbol = attributes.split('gene_name "')[1].split('"')[0]
            else:
                gene_symbol = gene_id_clean
            gene_mapping[gene_id_clean] = gene_symbol
    return gene_mapping

# -----------------------------
# Step 1: Load your dataset and extract gene IDs
# -----------------------------
adata_path = "/data/cellular_aging/preprocessed_batches/binned_sample.h5ad"
adata = sc.read_h5ad(adata_path)

# The dataset's gene identifiers (typically Ensembl IDs, versionless or not)
dataset_gene_ids = set(adata.var_names)
print(f"Dataset: Total unique gene IDs = {len(dataset_gene_ids)}")
print("Sample dataset gene IDs:", list(dataset_gene_ids)[:10])

# -----------------------------
# Step 2: Load your pretrained vocabulary and extract gene symbols
# -----------------------------
vocab_path = "/data/cellular_aging/references/scGPT_human_pretrained_model/vocab.json"
with open(vocab_path, "r") as f:
    vocab = json.load(f)
vocab_gene_symbols = set(vocab.keys())
print(f"Vocab: Total gene symbols in vocab.json = {len(vocab_gene_symbols)}")
print("Sample vocab gene symbols:", list(vocab_gene_symbols)[:10])

# -----------------------------
# Step 3: Extract gene mapping from a GTF file (e.g., Basic Annotation)
# -----------------------------
gtf_basic_path = "/data/cellular_aging/references/gencode.v47.basic.annotation.gtf.gz"
basic_mapping = extract_gene_mapping_from_gtf(gtf_basic_path)
basic_mapping_ids = set(basic_mapping.keys())
basic_mapping_symbols = set(basic_mapping.values())
print(f"GTF (Basic): Total gene IDs = {len(basic_mapping_ids)}")
print("Sample gene mapping from GTF (Basic):", list(basic_mapping_symbols)[:10])

# -----------------------------
# Step 4: Compare dataset gene IDs with the GTF mapping
# -----------------------------
common_dataset_basic = dataset_gene_ids.intersection(basic_mapping_ids)
missing_in_basic = dataset_gene_ids - basic_mapping_ids
print(f"Common gene IDs between dataset and GTF (Basic): {len(common_dataset_basic)}")
print(f"Missing gene IDs in GTF (Basic): {len(missing_in_basic)}")
print("Sample missing gene IDs:", list(missing_in_basic)[:10])

# -----------------------------
# Step 5: Compare vocab gene symbols with GTF gene symbols
# -----------------------------
common_vocab_basic = vocab_gene_symbols.intersection(basic_mapping_symbols)
print(f"Common gene symbols between vocab.json and GTF (Basic): {len(common_vocab_basic)}")
print("Sample common gene symbols:", list(common_vocab_basic)[:10])

# -----------------------------
# Optional: Repeat for the chr_patch_hapl_scaff GTF file
# -----------------------------
gtf_patch_path = "/data/cellular_aging/references/gencode.v47.chr_patch_hapl_scaff.annotation.gtf.gz"
patch_mapping = extract_gene_mapping_from_gtf(gtf_patch_path)
patch_mapping_ids = set(patch_mapping.keys())
patch_mapping_symbols = set(patch_mapping.values())
print(f"GTF (chr_patch_hapl_scaff): Total gene IDs = {len(patch_mapping_ids)}")
common_dataset_patch = dataset_gene_ids.intersection(patch_mapping_ids)
print(f"Common gene IDs between dataset and GTF (chr_patch_hapl_scaff): {len(common_dataset_patch)}")

# -----------------------------
# Summary of Comparisons
# -----------------------------
print("\nSummary of Comparisons:")
print(f"Dataset gene IDs: {len(dataset_gene_ids)}")
print(f"GTF (Basic) gene IDs: {len(basic_mapping_ids)}")
print(f"Common (Dataset ∩ GTF Basic): {len(common_dataset_basic)}")
print(f"Vocab gene symbols: {len(vocab_gene_symbols)}")
print(f"GTF (Basic) gene symbols: {len(basic_mapping_symbols)}")
print(f"Common (Vocab ∩ GTF Basic): {len(common_vocab_basic)}")


Dataset: Total unique gene IDs = 36161
Sample dataset gene IDs: ['ENSG00000251391', 'ENSG00000174032', 'ENSG00000117226', 'ENSG00000272906', 'ENSG00000284523', 'ENSG00000167528', 'ENSG00000132405', 'ENSG00000266714', 'ENSG00000277423', 'ENSG00000253702']
Vocab: Total gene symbols in vocab.json = 60697
Sample vocab gene symbols: ['OR8S1', 'LL0XNC01-7P3.1', 'AC142528.1', 'DENND1C', 'RP11-373A6.2', 'SPRR3', 'RN7SL300P', 'RP5-1171I10.5', 'TBCAP3', 'LINC00375']
GTF (Basic): Total gene IDs = 78724
Sample gene mapping from GTF (Basic): ['SPRR3', 'OR8S1', 'DENND1C', 'ENSG00000251391', 'RN7SL300P', 'TBCAP3', 'LINC00375', 'OR4K5', 'PPIAP21', 'ENSG00000272906']
Common gene IDs between dataset and GTF (Basic): 35213
Missing gene IDs in GTF (Basic): 948
Sample missing gene IDs: ['ENSG00000286755', 'ENSG00000254138', 'ENSG00000272218', 'ENSG00000254420', 'ENSG00000254290', 'ENSG00000237352', 'ENSG00000223458', 'ENSG00000273729', 'ENSG00000287211', 'ENSG00000286164']
Common gene symbols between vocab

In [11]:
import gzip
import json
import pandas as pd
import scanpy as sc

# ---------------------------
# Step 1: Load Dataset and Vocab
# ---------------------------

# Load your dataset (assumed to have Ensembl gene IDs in adata.var_names)
adata_path = "/data/cellular_aging/preprocessed_batches/binned_sample.h5ad"
adata = sc.read_h5ad(adata_path)
dataset_gene_ids = set(adata.var_names)
print(f"Dataset: Total unique gene IDs = {len(dataset_gene_ids)}")
print("Sample dataset gene IDs:", list(dataset_gene_ids)[:10])

# Load the pretrained vocabulary file (vocab.json)
vocab_path = "/data/cellular_aging/references/scGPT_human_pretrained_model/vocab.json"
with open(vocab_path, "r") as f:
    vocab = json.load(f)
vocab_gene_symbols = set(vocab.keys())
print(f"Vocab: Total gene symbols in vocab.json = {len(vocab_gene_symbols)}")
print("Sample vocab gene symbols:", list(vocab_gene_symbols)[:10])

# ---------------------------
# Step 2: Define a Function to Extract Gene Mapping from a GTF File
# ---------------------------
def extract_gene_mapping_from_gtf(gtf_file):
    """
    Extract a dictionary mapping Ensembl gene IDs (without version numbers)
    to gene symbols from a GTF file.
    """
    gene_mapping = {}
    with gzip.open(gtf_file, "rt") as f:
        for line in f:
            if line.startswith("#"):
                continue
            fields = line.strip().split("\t")
            if fields[2] != "gene":
                continue
            attributes = fields[-1]
            try:
                gene_id = attributes.split('gene_id "')[1].split('"')[0]
            except IndexError:
                continue
            # Remove version number (e.g., ENSG00000290825.2 -> ENSG00000290825)
            gene_id_clean = gene_id.split(".")[0]
            # Extract gene_name if available; else use gene_id as fallback
            if 'gene_name "' in attributes:
                gene_symbol = attributes.split('gene_name "')[1].split('"')[0]
            else:
                gene_symbol = gene_id_clean
            gene_mapping[gene_id_clean] = gene_symbol
    return gene_mapping

# ---------------------------
# Step 3: Extract Mappings from the Three GTF Files
# ---------------------------

# 1. Comprehensive gene annotation
gtf_comprehensive_path = "/data/cellular_aging/references/gencode.v47.annotation.gtf.gz"
mapping_comprehensive = extract_gene_mapping_from_gtf(gtf_comprehensive_path)
comp_ids = set(mapping_comprehensive.keys())
comp_symbols = set(mapping_comprehensive.values())
print(f"GTF (Comprehensive): Total gene IDs = {len(comp_ids)}")
print("Sample gene symbols (Comprehensive):", list(comp_symbols)[:10])

# 2. Basic gene annotation
gtf_basic_path = "/data/cellular_aging/references/gencode.v47.basic.annotation.gtf.gz"
mapping_basic = extract_gene_mapping_from_gtf(gtf_basic_path)
basic_ids = set(mapping_basic.keys())
basic_symbols = set(mapping_basic.values())
print(f"GTF (Basic): Total gene IDs = {len(basic_ids)}")
print("Sample gene symbols (Basic):", list(basic_symbols)[:10])

# 3. chr_patch_hapl_scaff annotation
gtf_patch_path = "/data/cellular_aging/references/gencode.v47.chr_patch_hapl_scaff.annotation.gtf.gz"
mapping_patch = extract_gene_mapping_from_gtf(gtf_patch_path)
patch_ids = set(mapping_patch.keys())
patch_symbols = set(mapping_patch.values())
print(f"GTF (chr_patch_hapl_scaff): Total gene IDs = {len(patch_ids)}")
print("Sample gene symbols (chr_patch_hapl_scaff):", list(patch_symbols)[:10])

# ---------------------------
# Step 4: Compare Dataset Gene IDs with Each GTF Mapping
# ---------------------------

# For Comprehensive mapping:
common_comp = dataset_gene_ids.intersection(comp_ids)
missing_comp = dataset_gene_ids - comp_ids
print(f"\nCommon gene IDs (Dataset ∩ Comprehensive): {len(common_comp)}")
print(f"Missing gene IDs in Comprehensive: {len(missing_comp)}")
print("Sample missing (Comprehensive):", list(missing_comp)[:10])

# For Basic mapping:
common_basic = dataset_gene_ids.intersection(basic_ids)
missing_basic = dataset_gene_ids - basic_ids
print(f"\nCommon gene IDs (Dataset ∩ Basic): {len(common_basic)}")
print(f"Missing gene IDs in Basic: {len(missing_basic)}")
print("Sample missing (Basic):", list(missing_basic)[:10])

# For Patch mapping:
common_patch = dataset_gene_ids.intersection(patch_ids)
missing_patch = dataset_gene_ids - patch_ids
print(f"\nCommon gene IDs (Dataset ∩ chr_patch_hapl_scaff): {len(common_patch)}")
print(f"Missing gene IDs in chr_patch_hapl_scaff: {len(missing_patch)}")
print("Sample missing (Patch):", list(missing_patch)[:10])

# ---------------------------
# Step 5: Compare Vocab Gene Symbols with Each GTF Mapping's Gene Symbols
# ---------------------------
common_vocab_comp = vocab_gene_symbols.intersection(comp_symbols)
common_vocab_basic = vocab_gene_symbols.intersection(basic_symbols)
common_vocab_patch = vocab_gene_symbols.intersection(patch_symbols)

print(f"\nCommon gene symbols (Vocab ∩ Comprehensive): {len(common_vocab_comp)}")
print(f"Common gene symbols (Vocab ∩ Basic): {len(common_vocab_basic)}")
print(f"Common gene symbols (Vocab ∩ chr_patch_hapl_scaff): {len(common_vocab_patch)}")

# ---------------------------
# Step 6: Summary of Comparisons
# ---------------------------
print("\nSummary of Comparisons:")
print(f"Dataset gene IDs: {len(dataset_gene_ids)}")
print(f"GTF (Comprehensive) gene IDs: {len(comp_ids)}")
print(f"GTF (Basic) gene IDs: {len(basic_ids)}")
print(f"GTF (chr_patch_hapl_scaff) gene IDs: {len(patch_ids)}")
print(f"Common (Dataset ∩ Comprehensive): {len(common_comp)}")
print(f"Common (Dataset ∩ Basic): {len(common_basic)}")
print(f"Common (Dataset ∩ chr_patch_hapl_scaff): {len(common_patch)}")
print(f"Vocab gene symbols: {len(vocab_gene_symbols)}")
print(f"Common gene symbols (Vocab ∩ Comprehensive): {len(common_vocab_comp)}")
print(f"Common gene symbols (Vocab ∩ Basic): {len(common_vocab_basic)}")
print(f"Common gene symbols (Vocab ∩ chr_patch_hapl_scaff): {len(common_vocab_patch)}")


Dataset: Total unique gene IDs = 36161
Sample dataset gene IDs: ['ENSG00000251391', 'ENSG00000174032', 'ENSG00000117226', 'ENSG00000272906', 'ENSG00000284523', 'ENSG00000167528', 'ENSG00000132405', 'ENSG00000266714', 'ENSG00000277423', 'ENSG00000253702']
Vocab: Total gene symbols in vocab.json = 60697
Sample vocab gene symbols: ['OR8S1', 'LL0XNC01-7P3.1', 'AC142528.1', 'DENND1C', 'RP11-373A6.2', 'SPRR3', 'RN7SL300P', 'RP5-1171I10.5', 'TBCAP3', 'LINC00375']
GTF (Comprehensive): Total gene IDs = 78724
Sample gene symbols (Comprehensive): ['SPRR3', 'OR8S1', 'DENND1C', 'ENSG00000251391', 'RN7SL300P', 'TBCAP3', 'LINC00375', 'OR4K5', 'PPIAP21', 'ENSG00000272906']
GTF (Basic): Total gene IDs = 78724
Sample gene symbols (Basic): ['SPRR3', 'OR8S1', 'DENND1C', 'ENSG00000251391', 'RN7SL300P', 'TBCAP3', 'LINC00375', 'OR4K5', 'PPIAP21', 'ENSG00000272906']
GTF (chr_patch_hapl_scaff): Total gene IDs = 86402
Sample gene symbols (chr_patch_hapl_scaff): ['SPRR3', 'OR8S1', 'DENND1C', 'LINC00375', 'OR4K5'

In [12]:
import gzip
import json
import pandas as pd
import scanpy as sc

# ---------------------------
# Step 1: Load Dataset and Pretrained Vocab
# ---------------------------

# Load your dataset (AnnData file)
adata_path = "/data/cellular_aging/preprocessed_batches/binned_sample.h5ad"
adata = sc.read_h5ad(adata_path)
dataset_gene_ids = set(adata.var_names)
print(f"Dataset: Total unique gene IDs = {len(dataset_gene_ids)}")
print("Sample dataset gene IDs:", list(dataset_gene_ids)[:10])

# Load the pretrained vocabulary file (gene symbols to token IDs)
vocab_path = "/data/cellular_aging/references/scGPT_human_pretrained_model/vocab.json"
with open(vocab_path, "r") as f:
    vocab = json.load(f)
vocab_gene_symbols = set(vocab.keys())
print(f"Vocab: Total gene symbols in vocab.json = {len(vocab_gene_symbols)}")
print("Sample vocab gene symbols:", list(vocab_gene_symbols)[:10])

# ---------------------------
# Step 2: Define Function to Extract Gene Mapping from a GTF File
# ---------------------------
def extract_gene_mapping_from_gtf(gtf_file):
    """
    Extracts a dictionary mapping Ensembl gene IDs (version numbers removed)
    to gene symbols from a GTF file.
    """
    gene_mapping = {}
    with gzip.open(gtf_file, "rt") as f:
        for line in f:
            if line.startswith("#"):
                continue  # Skip header lines
            fields = line.strip().split("\t")
            if fields[2] != "gene":
                continue  # Process only gene entries
            attributes = fields[-1]
            try:
                gene_id = attributes.split('gene_id "')[1].split('"')[0]
            except IndexError:
                continue
            gene_id_clean = gene_id.split(".")[0]  # Remove version number
            # Extract gene_name if available; else use gene_id_clean as fallback
            if 'gene_name "' in attributes:
                gene_symbol = attributes.split('gene_name "')[1].split('"')[0]
            else:
                gene_symbol = gene_id_clean
            gene_mapping[gene_id_clean] = gene_symbol
    return gene_mapping

# ---------------------------
# Step 3: Extract Mappings from Three GTF Files
# ---------------------------

# 3.1 Comprehensive gene annotation
gtf_comprehensive_path = "/data/cellular_aging/references/gencode.v47.annotation.gtf.gz"
mapping_comprehensive = extract_gene_mapping_from_gtf(gtf_comprehensive_path)
comp_ids = set(mapping_comprehensive.keys())
comp_symbols = set(mapping_comprehensive.values())
print(f"\nGTF (Comprehensive): Total gene IDs = {len(comp_ids)}")
print("Sample gene symbols (Comprehensive):", list(comp_symbols)[:10])

# 3.2 Basic gene annotation
gtf_basic_path = "/data/cellular_aging/references/gencode.v47.basic.annotation.gtf.gz"
mapping_basic = extract_gene_mapping_from_gtf(gtf_basic_path)
basic_ids = set(mapping_basic.keys())
basic_symbols = set(mapping_basic.values())
print(f"\nGTF (Basic): Total gene IDs = {len(basic_ids)}")
print("Sample gene symbols (Basic):", list(basic_symbols)[:10])

# 3.3 chr_patch_hapl_scaff annotation
gtf_patch_path = "/data/cellular_aging/references/gencode.v47.chr_patch_hapl_scaff.annotation.gtf.gz"
mapping_patch = extract_gene_mapping_from_gtf(gtf_patch_path)
patch_ids = set(mapping_patch.keys())
patch_symbols = set(mapping_patch.values())
print(f"\nGTF (chr_patch_hapl_scaff): Total gene IDs = {len(patch_ids)}")
print("Sample gene symbols (chr_patch_hapl_scaff):", list(patch_symbols)[:10])

# ---------------------------
# Step 4: Compare Dataset Gene IDs with Each GTF Mapping
# ---------------------------

# Comparison with Comprehensive mapping:
common_comp = dataset_gene_ids.intersection(comp_ids)
missing_comp = dataset_gene_ids - comp_ids
print(f"\nCommon gene IDs (Dataset ∩ Comprehensive): {len(common_comp)}")
print(f"Missing gene IDs in Comprehensive: {len(missing_comp)}")
print("Sample missing (Comprehensive):", list(missing_comp)[:10])

# Comparison with Basic mapping:
common_basic = dataset_gene_ids.intersection(basic_ids)
missing_basic = dataset_gene_ids - basic_ids
print(f"\nCommon gene IDs (Dataset ∩ Basic): {len(common_basic)}")
print(f"Missing gene IDs in Basic: {len(missing_basic)}")
print("Sample missing (Basic):", list(missing_basic)[:10])

# Comparison with Patch mapping:
common_patch = dataset_gene_ids.intersection(patch_ids)
missing_patch = dataset_gene_ids - patch_ids
print(f"\nCommon gene IDs (Dataset ∩ chr_patch_hapl_scaff): {len(common_patch)}")
print(f"Missing gene IDs in chr_patch_hapl_scaff: {len(missing_patch)}")
print("Sample missing (Patch):", list(missing_patch)[:10])

# ---------------------------
# Step 5: Compare Vocab Gene Symbols with Each GTF Mapping's Gene Symbols
# ---------------------------

common_vocab_comp = vocab_gene_symbols.intersection(comp_symbols)
common_vocab_basic = vocab_gene_symbols.intersection(basic_symbols)
common_vocab_patch = vocab_gene_symbols.intersection(patch_symbols)

print(f"\nCommon gene symbols (Vocab ∩ Comprehensive): {len(common_vocab_comp)}")
print(f"Common gene symbols (Vocab ∩ Basic): {len(common_vocab_basic)}")
print(f"Common gene symbols (Vocab ∩ chr_patch_hapl_scaff): {len(common_vocab_patch)}")
print("Sample common gene symbols (e.g., Basic):", list(common_vocab_basic)[:10])

# ---------------------------
# Step 6: Print a Summary of Comparisons
# ---------------------------
print("\nSummary of Comparisons:")
print(f"Dataset gene IDs: {len(dataset_gene_ids)}")
print(f"GTF (Comprehensive) gene IDs: {len(comp_ids)}")
print(f"GTF (Basic) gene IDs: {len(basic_ids)}")
print(f"GTF (chr_patch_hapl_scaff) gene IDs: {len(patch_ids)}")
print(f"Common (Dataset ∩ Comprehensive): {len(common_comp)}")
print(f"Common (Dataset ∩ Basic): {len(common_basic)}")
print(f"Common (Dataset ∩ chr_patch_hapl_scaff): {len(common_patch)}")
print(f"Vocab gene symbols: {len(vocab_gene_symbols)}")
print(f"Common gene symbols (Vocab ∩ Comprehensive): {len(common_vocab_comp)}")
print(f"Common gene symbols (Vocab ∩ Basic): {len(common_vocab_basic)}")
print(f"Common gene symbols (Vocab ∩ chr_patch_hapl_scaff): {len(common_vocab_patch)}")


Dataset: Total unique gene IDs = 36161
Sample dataset gene IDs: ['ENSG00000251391', 'ENSG00000174032', 'ENSG00000117226', 'ENSG00000272906', 'ENSG00000284523', 'ENSG00000167528', 'ENSG00000132405', 'ENSG00000266714', 'ENSG00000277423', 'ENSG00000253702']
Vocab: Total gene symbols in vocab.json = 60697
Sample vocab gene symbols: ['OR8S1', 'LL0XNC01-7P3.1', 'AC142528.1', 'DENND1C', 'RP11-373A6.2', 'SPRR3', 'RN7SL300P', 'RP5-1171I10.5', 'TBCAP3', 'LINC00375']

GTF (Comprehensive): Total gene IDs = 78724
Sample gene symbols (Comprehensive): ['SPRR3', 'OR8S1', 'DENND1C', 'ENSG00000251391', 'RN7SL300P', 'TBCAP3', 'LINC00375', 'OR4K5', 'PPIAP21', 'ENSG00000272906']

GTF (Basic): Total gene IDs = 78724
Sample gene symbols (Basic): ['SPRR3', 'OR8S1', 'DENND1C', 'ENSG00000251391', 'RN7SL300P', 'TBCAP3', 'LINC00375', 'OR4K5', 'PPIAP21', 'ENSG00000272906']

GTF (chr_patch_hapl_scaff): Total gene IDs = 86402
Sample gene symbols (chr_patch_hapl_scaff): ['SPRR3', 'OR8S1', 'DENND1C', 'LINC00375', 'OR4